In [9]:
import pandas as pd
import numpy as np
from math import ceil

In [10]:
str_date = "2022-09-15"
df = pd.read_csv("Honolulu_new_" + str_date + ".csv")

In [11]:
def roundU(diff, t, tick, b):
    if diff >= 0:
        return diff * (1 - b * int(t / tick))
    else:
        return diff * (1 + b * int(t / tick))

def accurateU(diff, t, tick, b):
    if diff >= 0:
        return diff * (1 - b * t / tick)
    else:
        return diff * (1 + b * t / tick)

In [12]:
df["player.english_dropout_elpased"].fillna(value=0, inplace=True)
df["player.dropout_price"] = np.nan
df["player.dropout_price.accurate"] = np.nan
df["player.dropout_payoff"] = np.nan
df["player.dropout_payoff.accurate"] = np.nan

In [13]:
for i in range(len(df)):
    price_start = df["subsession.price_start"].iat[i]
    price_tick = df["session.config.price_tick"].iat[i]
    item_value = df["player.item_value"].iat[i]
    discount_b = df["session.config.discount_b"].iat[i]
    dutch_time_elapsed = df["group.dutch_time_elapsed"].iat[i]
    english_time_elapsed = df["player.english_dropout_elpased"].iat[i]
    if df["player.contest_status"].iat[i] != 1:
        df["player.dropout_price"].iat[i] = price_start - int(dutch_time_elapsed / price_tick)
        df["player.dropout_price.accurate"].iat[i] = price_start - dutch_time_elapsed / price_tick
        df["player.dropout_payoff"].iat[i] = roundU(item_value - df["player.dropout_price"].iat[i], dutch_time_elapsed, price_tick, discount_b)
        df["player.dropout_payoff.accurate"].iat[i] = accurateU(item_value - df["player.dropout_price.accurate"].iat[i], dutch_time_elapsed, price_tick, discount_b)
    else:
        df["player.dropout_price"].iat[i] = price_start + ceil((english_time_elapsed - dutch_time_elapsed) / price_tick) # use int() for data before 2022/10/26, use ceil() otherwise
        df["player.dropout_price.accurate"].iat[i] = price_start + (english_time_elapsed - dutch_time_elapsed) / price_tick
        df["player.dropout_payoff"].iat[i] = roundU(item_value - df["player.dropout_price"].iat[i], dutch_time_elapsed + english_time_elapsed, price_tick, discount_b)
        df["player.dropout_payoff.accurate"].iat[i] = accurateU(item_value - df["player.dropout_price.accurate"].iat[i], dutch_time_elapsed + english_time_elapsed, price_tick, discount_b)

In [14]:
df["predict.player.english_dropout_elapsed"].fillna(value=0, inplace=True)
df["predict.player.dropout_price"] = np.nan
df["predict.player.dropout_price.accurate"] = np.nan
df["predict.player.dropout_payoff"] = np.nan
df["predict.player.dropout_payoff.accurate"] = np.nan

In [15]:
for i in range(len(df)):
    price_start = df["subsession.price_start"].iat[i]
    price_tick = df["session.config.price_tick"].iat[i]
    item_value = df["player.item_value"].iat[i]
    discount_b = df["session.config.discount_b"].iat[i]
    dutch_time_elapsed = df["predict.group.dutch_time_elapsed"].iat[i]
    english_time_elapsed = df["predict.player.english_dropout_elapsed"].iat[i]
    if df["predict.player.contest_status"].iat[i] != 1:
        df["predict.player.dropout_price"].iat[i] = price_start - ceil(dutch_time_elapsed / price_tick)
        df["predict.player.dropout_price.accurate"].iat[i] = price_start - dutch_time_elapsed / price_tick
        df["predict.player.dropout_payoff"].iat[i] = roundU(item_value - df["predict.player.dropout_price"].iat[i], dutch_time_elapsed, price_tick, discount_b)
        df["predict.player.dropout_payoff.accurate"].iat[i] = accurateU(item_value - df["predict.player.dropout_price.accurate"].iat[i], dutch_time_elapsed, price_tick, discount_b)
    else:
        df["predict.player.dropout_price"].iat[i] = price_start + int((english_time_elapsed - dutch_time_elapsed) / price_tick) # use int() for data before 2022/10/26, use ceil() otherwise
        df["predict.player.dropout_price.accurate"].iat[i] = price_start + (english_time_elapsed - dutch_time_elapsed) / price_tick
        df["predict.player.dropout_payoff"].iat[i] = roundU(item_value - df["predict.player.dropout_price"].iat[i], dutch_time_elapsed + english_time_elapsed, price_tick, discount_b)
        df["predict.player.dropout_payoff.accurate"].iat[i] = accurateU(item_value - df["predict.player.dropout_price.accurate"].iat[i], dutch_time_elapsed + english_time_elapsed, price_tick, discount_b)

In [16]:
df.to_csv("Honolulu_rounded_and_accurate_" + str_date + ".csv", header=True, index=False)